In [1]:
import os
import time
import glob
import re
import pandas as pd
from functools import reduce
from klepto.archives import dir_archive

import numpy as np
from tqdm.auto import tqdm
import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea.analysis_objects import JaggedCandidateArray
from coffea.btag_tools import BTagScaleFactor
from coffea import hist
import pandas as pd
import uproot_methods
import uproot
import awkward
import copy

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from Tools.config_helpers import *
from Tools.helpers import mergeArray, mt

from Tools.objects import Collections
from Tools.cutflow import Cutflow

# This just tells matplotlib not to open any
# interactive windows.
matplotlib.use('Agg')

In [2]:
def pad_and_flatten(val): 
    try:
        return val.pad(1, clip=True).fillna(0.).flatten()
    except AttributeError:
        return val.flatten()

In [3]:
# Imports and defintions for the actual analysis to run

from Tools.WH_objects       import *
from Tools.WH_scalefactors  import LeptonSF
from Tools.WH_deepAK8       import getWTagSF
from Tools.WH_signalWeights import getSignalWeight
from Tools.WH_samples       import * 

year = 2018

In [4]:
processesList = ['full_sim', 'WJets', 'TTJets', 'ST', 'ttW', 'WW', 'ZNuNu', 'QCD', 'Data']
lumis       = {2016: 36., 2017: 41.5, 2018: 60.}

In [5]:
class analysisProcessor(processor.ProcessorABC):
    """Processor used for running the analysis"""
    def __init__(self):
        
        # we can use a large number of bins and rebin later
        dataset_axis        = hist.Cat("objecto",   "Object")
        pt_axis             = hist.Bin("pt",        r"$p_{T}$ (GeV)", 1000, 0, 1000)
        p_axis              = hist.Bin("p",         r"$p$ (GeV)", 1000, 0, 2500)
        ht_axis             = hist.Bin("ht",        r"$H_{T}$ (GeV)", 500, 0, 5000)
        mass_axis           = hist.Bin("mass",      r"M (GeV)", 1000, 0, 2000)
        eta_axis            = hist.Bin("eta",       r"$\eta$", 60, -5.5, 5.5)
        delta_axis          = hist.Bin("delta",     r"$\delta$", 100,0,10 )
        score_axis          = hist.Bin("score",     r"$score$", 100,0,1 )
        multiplicity_axis   = hist.Bin("multiplicity",         r"N", 20, -0.5, 19.5)
        norm_axis           = hist.Bin("norm",         r"N", 25, 0, 1)

        self._accumulator = processor.dict_accumulator({
            "lead_AK8_pts1":      hist.Hist("Counts", dataset_axis, pt_axis),
            "sublead_AK8_pts1":      hist.Hist("Counts", dataset_axis, pt_axis),
            
            "lead_AK8_msoftdrops1":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops1":  hist.Hist("Counts", dataset_axis, mass_axis),
            
            "lead_AK8_pts2":      hist.Hist("Counts", dataset_axis, pt_axis),
            "sublead_AK8_pts2":      hist.Hist("Counts", dataset_axis, pt_axis),
            
            "lead_AK8_msoftdrops2":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops2":  hist.Hist("Counts", dataset_axis, mass_axis),

            
            "lead_AK8_msoftdrops3":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops3":  hist.Hist("Counts", dataset_axis, mass_axis),
             
            
            "lead_AK8_msoftdrops4":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops4":  hist.Hist("Counts", dataset_axis, mass_axis),
            
            
            "lead_AK8_msoftdrops5":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops5":  hist.Hist("Counts", dataset_axis, mass_axis),
            
         
            "lead_AK8_msoftdrops6":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops6":  hist.Hist("Counts", dataset_axis, mass_axis),
            
            
            "lead_AK8_pts7":      hist.Hist("Counts", dataset_axis, pt_axis),
            "sublead_AK8_pts7":      hist.Hist("Counts", dataset_axis, pt_axis),
            
            "lead_AK8_msoftdrops7":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops7":  hist.Hist("Counts", dataset_axis, mass_axis),
            
            
            "lead_AK8_pts8":      hist.Hist("Counts", dataset_axis, pt_axis),
            "sublead_AK8_pts8":      hist.Hist("Counts", dataset_axis, pt_axis),
            
            "lead_AK8_msoftdrops8":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops8":  hist.Hist("Counts", dataset_axis, mass_axis),
            
            
            "lead_AK8_pts9":      hist.Hist("Counts", dataset_axis, pt_axis),
            "sublead_AK8_pts9":      hist.Hist("Counts", dataset_axis, pt_axis),
            
            "lead_AK8_msoftdrops9":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops9":  hist.Hist("Counts", dataset_axis, mass_axis),
            
            
            "lead_AK8_pts10":      hist.Hist("Counts", dataset_axis, pt_axis),
            "sublead_AK8_pts10":      hist.Hist("Counts", dataset_axis, pt_axis),
            
            "lead_AK8_msoftdrops10":  hist.Hist("Counts", dataset_axis, mass_axis),
            "sublead_AK8_msoftdrops10":  hist.Hist("Counts", dataset_axis, mass_axis),
             
            'full_sim':         processor.defaultdict_accumulator(int),
            'WJets':            processor.defaultdict_accumulator(int),
            'QCD':              processor.defaultdict_accumulator(int),
            'TTJets':           processor.defaultdict_accumulator(int),
            'ZNuNu':            processor.defaultdict_accumulator(int),
            'ST':               processor.defaultdict_accumulator(int),
            'ST_tW':            processor.defaultdict_accumulator(int),
            'ST_tChannel':      processor.defaultdict_accumulator(int),
            'ST_sChannel':      processor.defaultdict_accumulator(int),
            'ttW':              processor.defaultdict_accumulator(int),
            'ttZ':              processor.defaultdict_accumulator(int),
            'WW':               processor.defaultdict_accumulator(int),
            'WZ/ZZ':            processor.defaultdict_accumulator(int),
            'LL':               processor.defaultdict_accumulator(int),
            'Data':             processor.defaultdict_accumulator(int),
            'totalEvents':      processor.defaultdict_accumulator(int),
            'test1':            processor.defaultdict_accumulator(float),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        """
        Processing function. This is where the actual analysis happens.
        """
        output = self.accumulator.identity()
        dataset = df["dataset"]
        cfg = loadConfig()
        
        ## MET -> can switch to puppi MET
        met_pt  = df["MET_pt"]
        met_phi = df["MET_phi"] 
        
        filters  = getFilters(df, year=year, dataset=dataset)
        triggers = getTriggers(df, year=year, dataset=dataset)
        
        muon_veto     = getMuons(df, WP='veto')
        electron_veto = getElectrons(df, WP='veto')
        muon_med      = getMuons(df, WP='medium')
        electron_med  = getElectrons(df, WP='medium')
        fatjet1        = getFatJets(df, ptcorr = '', masscorr = '')
        fatjet2        = getFatJets(df, ptcorr = '_nom', masscorr = '_nom')
        fatjet3        = getFatJets(df, ptcorr = '_nom', masscorr = '_jmsUp')
        fatjet4        = getFatJets(df, ptcorr = '_nom', masscorr = '_jmsDown')
        fatjet5        = getFatJets(df, ptcorr = '_nom', masscorr = '_jmrUp')
        fatjet6        = getFatJets(df, ptcorr = '_nom', masscorr = '_jmrDown')
        fatjet7        = getFatJets(df, ptcorr = '_jerUp', masscorr = '_jerUp')
        fatjet8        = getFatJets(df, ptcorr = '_jerDown', masscorr = '_jerDown')
        fatjet9        = getFatJets(df, ptcorr = '_jesTotalUp', masscorr = '_jesTotalUp')
        fatjet10       = getFatJets(df, ptcorr = '_jesTotalDown', masscorr = '_jesTotalDown')
        jet           = getJets(df)

        sf = LeptonSF(year=year)
        leptonSF = sf.get(electron_veto, muon_veto)
        
        # clean AK4 jets
        skimjet   = jet[(jet.pt>30) & (jet.jetId>1) & (abs(jet.eta)<2.4)]
        jet       = jet[~jet.match(muon_veto, deltaRCut=0.4)] # remove jets that overlap with muons
        jet       = jet[~jet.match(electron_veto, deltaRCut=0.4)] # remove jets that overlap with electrons
        jet       = jet[jet.pt.argsort(ascending=False)] # sort the jets
        extrajet1  = jet[~jet.match(fatjet1, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet2  = jet[~jet.match(fatjet2, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet3  = jet[~jet.match(fatjet3, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet4  = jet[~jet.match(fatjet4, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet5  = jet[~jet.match(fatjet5, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet6  = jet[~jet.match(fatjet6, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet7  = jet[~jet.match(fatjet7, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet8  = jet[~jet.match(fatjet8, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet9  = jet[~jet.match(fatjet9, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        extrajet10 = jet[~jet.match(fatjet10, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        btag      = getBTags(jet, year=year)
        
        # some gymnastics with AK8 jets
        leadingFatJets1 = fatjet1[:,:2]
        leadingFatJets2 = fatjet2[:,:2]
        leadingFatJets3 = fatjet3[:,:2]
        leadingFatJets4 = fatjet4[:,:2]
        leadingFatJets5 = fatjet5[:,:2]
        leadingFatJets6 = fatjet6[:,:2]
        leadingFatJets7 = fatjet7[:,:2]
        leadingFatJets8 = fatjet8[:,:2]
        leadingFatJets9 = fatjet9[:,:2]
        leadingFatJets10 = fatjet10[:,:2]
        
        leadingFatJet1 = fatjet1[:,:1]
        subleadingFatJet1 = fatjet1[:,1:2]
        
        leadingFatJet2 = fatjet2[:,:1]
        subleadingFatJet2 = fatjet2[:,1:2]
        
        leadingFatJet3 = fatjet3[:,:1]
        subleadingFatJet3 = fatjet3[:,1:2]
        
        leadingFatJet4 = fatjet4[:,:1]
        subleadingFatJet4 = fatjet4[:,1:2]
        
        leadingFatJet5 = fatjet5[:,:1]
        subleadingFatJet5 = fatjet5[:,1:2]
        
        leadingFatJet6 = fatjet6[:,:1]
        subleadingFatJet6 = fatjet6[:,1:2]
        
        leadingFatJet7 = fatjet7[:,:1]
        subleadingFatJet7 = fatjet7[:,1:2]
        
        leadingFatJet8 = fatjet8[:,:1]
        subleadingFatJet8 = fatjet8[:,1:2]
        
        leadingFatJet9 = fatjet9[:,:1]
        subleadingFatJet9 = fatjet9[:,1:2]
        
        leadingFatJet10 = fatjet10[:,:1]
        subleadingFatJet10 = fatjet10[:,1:2]
        
        difatjet1 = leadingFatJets1.choose(2)
        dphiDiFatJet1 = np.arccos(np.cos(difatjet1.i0.phi-difatjet1.i1.phi))
        min_dphiFatJetMet41 = np.arccos(np.cos(fatjet1[:,:4].phi-met_phi)).min()
        
        difatjet2 = leadingFatJets2.choose(2)
        dphiDiFatJet2 = np.arccos(np.cos(difatjet2.i0.phi-difatjet2.i1.phi))
        min_dphiFatJetMet42 = np.arccos(np.cos(fatjet2[:,:4].phi-met_phi)).min()
        
        difatjet3 = leadingFatJets3.choose(2)
        dphiDiFatJet3 = np.arccos(np.cos(difatjet3.i0.phi-difatjet3.i1.phi))
        min_dphiFatJetMet43 = np.arccos(np.cos(fatjet3[:,:4].phi-met_phi)).min()
        
        difatjet4 = leadingFatJets4.choose(2)
        dphiDiFatJet4 = np.arccos(np.cos(difatjet4.i0.phi-difatjet4.i1.phi))
        min_dphiFatJetMet44 = np.arccos(np.cos(fatjet4[:,:4].phi-met_phi)).min()
        
        difatjet5 = leadingFatJets5.choose(2)
        dphiDiFatJet5 = np.arccos(np.cos(difatjet5.i0.phi-difatjet5.i1.phi))
        min_dphiFatJetMet45 = np.arccos(np.cos(fatjet5[:,:4].phi-met_phi)).min()
        
        difatjet6 = leadingFatJets6.choose(2)
        dphiDiFatJet6 = np.arccos(np.cos(difatjet6.i0.phi-difatjet6.i1.phi))
        min_dphiFatJetMet46 = np.arccos(np.cos(fatjet6[:,:4].phi-met_phi)).min()
        
        difatjet7 = leadingFatJets7.choose(2)
        dphiDiFatJet7 = np.arccos(np.cos(difatjet7.i0.phi-difatjet7.i1.phi))
        min_dphiFatJetMet47 = np.arccos(np.cos(fatjet7[:,:4].phi-met_phi)).min()
        
        difatjet8 = leadingFatJets8.choose(2)
        dphiDiFatJet8 = np.arccos(np.cos(difatjet8.i0.phi-difatjet8.i1.phi))
        min_dphiFatJetMet48 = np.arccos(np.cos(fatjet8[:,:4].phi-met_phi)).min()
        
        difatjet9 = leadingFatJets9.choose(2)
        dphiDiFatJet9 = np.arccos(np.cos(difatjet9.i0.phi-difatjet9.i1.phi))
        min_dphiFatJetMet49 = np.arccos(np.cos(fatjet9[:,:4].phi-met_phi)).min()
        
        difatjet10 = leadingFatJets10.choose(2)
        dphiDiFatJet10 = np.arccos(np.cos(difatjet10.i0.phi-difatjet10.i1.phi))
        min_dphiFatJetMet410 = np.arccos(np.cos(fatjet10[:,:4].phi-met_phi)).min()
        
        #H tags
        
        htag1 = getHTags(fatjet1, year=year)        
        lead_htag1 = htag1[htag1.pt.argmax()]
                
        mth1 = mt(htag1.pt, htag1.phi, met_pt, met_phi)

        ht = jet.pt.sum()

        htag_sel1 = ( htag1.counts>0 & (abs(htag1.msoftdrop-125)<25).any())

        
        htag2 = getHTags(fatjet2, year=year)        
        lead_htag2 = htag2[htag2.pt.argmax()]
                
        mth2 = mt(htag2.pt, htag2.phi, met_pt, met_phi)

        htag_sel2 = ( htag2.counts>0 & (abs(htag2.msoftdrop-125)<25).any())
        
        
        htag3 = getHTags(fatjet3, year=year)        
        lead_htag3 = htag3[htag3.pt.argmax()]
                
        mth3 = mt(htag3.pt, htag3.phi, met_pt, met_phi)

        htag_sel3 = ( htag3.counts>0 & (abs(htag3.msoftdrop-125)<25).any())
        
        
        htag4 = getHTags(fatjet4, year=year)        
        lead_htag4 = htag4[htag4.pt.argmax()]
                
        mth4 = mt(htag4.pt, htag4.phi, met_pt, met_phi)

        htag_sel4 = ( htag4.counts>0 & (abs(htag4.msoftdrop-125)<25).any())
        
        
        htag5 = getHTags(fatjet5, year=year)        
        lead_htag5 = htag5[htag5.pt.argmax()]
                
        mth5 = mt(htag5.pt, htag5.phi, met_pt, met_phi)

        htag_sel5 = ( htag5.counts>0 & (abs(htag5.msoftdrop-125)<25).any())
        
        
        htag6 = getHTags(fatjet6, year=year)        
        lead_htag6 = htag6[htag6.pt.argmax()]
                
        mth6 = mt(htag6.pt, htag6.phi, met_pt, met_phi)

        htag_sel6 = ( htag6.counts>0 & (abs(htag6.msoftdrop-125)<25).any())
        
        
        htag7 = getHTags(fatjet7, year=year)        
        lead_htag7 = htag7[htag7.pt.argmax()]
                
        mth7 = mt(htag7.pt, htag7.phi, met_pt, met_phi)

        htag_sel7 = ( htag7.counts>0 & (abs(htag7.msoftdrop-125)<25).any())
        
        
        htag8 = getHTags(fatjet8, year=year)        
        lead_htag8 = htag8[htag8.pt.argmax()]
                
        mth8 = mt(htag8.pt, htag8.phi, met_pt, met_phi)

        htag_sel8 = ( htag8.counts>0 & (abs(htag8.msoftdrop-125)<25).any())
        
        
        htag9 = getHTags(fatjet9, year=year)        
        lead_htag9 = htag9[htag9.pt.argmax()]
                
        mth9 = mt(htag9.pt, htag9.phi, met_pt, met_phi)

        htag_sel9 = ( htag9.counts>0 & (abs(htag9.msoftdrop-125)<25).any())
        
        
        htag10 = getHTags(fatjet10, year=year)        
        lead_htag10 = htag10[htag10.pt.argmax()]
                
        mth10 = mt(htag10.pt, htag10.phi, met_pt, met_phi)

        htag_sel10 = ( htag10.counts>0 & (abs(htag10.msoftdrop-125)<25).any())

        
        #W tags
        wtag = getWTags(fatjet1, year=year)
        wtag = wtag[~wtag.match(htag1, deltaRCut=0.8)]
        lead_wtag = wtag[wtag.pt.argmax()]
        
        if dataset.lower().count('data')==0:
            GenW = getGenW(df)
            wtag_SF = getWTagSF(wtag, GenW, year=year)
        
        wh = lead_htag1.cross(lead_wtag)
        wh_deltaPhi = np.arccos(wh.i0.phi - wh.i1.phi)
        wh_deltaR = wh.i0.p4.delta_r(wh.i1.p4)
        
        wtag_sel = ( wtag.counts>0 & (abs(wtag.msoftdrop-80)<30).any())
        #stitchVar = 1 if dataset=='Data' else df["stitch"]
        
        ## define selections (maybe move to a different file at some point)
               
        output['totalEvents']['all'] += len(df['weight'])
        
        # Cutflow
        processes   = processesList
        weight      = np.ones(len(df['weight'])) if dataset=='Data' else df['weight']*df['puWeight']*leptonSF*wtag_SF
        lumis       = {2016: 36., 2017: 41.5, 2018: 60.}
        cfg['lumi'] = 1 if dataset=='Data' else lumis[year]
        fullweight  = weight*cfg['lumi']
        
        cutflow1 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow1.addRow( 'filters',    filters )
        cutflow1.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow1.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow1.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow1.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline1 = copy.deepcopy(cutflow1.selection)
        
        cutflow1.addRow( 'N_fatjet>1',      (fatjet1.counts>1) )
        cutflow1.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet41>0.5))
        cutflow1.addRow( 'dphiDiFatJet', (dphiDiFatJet1<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets

        vetoQCD1 = copy.deepcopy(cutflow1.selection)
        
        cutflow1.addRow( 'N_htag>0',     (htag_sel1))
        cutflow1.addRow( 'njet veto',     (extrajet1.counts<2))
        cutflow1.addRow( 'minmth>200',   (mth1.min()>200) )

        htag_selection1 = copy.deepcopy(cutflow1.selection)
        

        cutflow2 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow2.addRow( 'filters',    filters )
        cutflow2.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow2.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow2.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow2.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline2 = copy.deepcopy(cutflow2.selection)
        
        cutflow2.addRow( 'N_fatjet>1',      (fatjet2.counts>1) )
        cutflow2.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet42>0.5))
        cutflow2.addRow( 'dphiDiFatJet', (dphiDiFatJet2<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD2 = copy.deepcopy(cutflow2.selection)
        
        cutflow2.addRow( 'N_htag>0',     (htag_sel2))
        cutflow2.addRow( 'njet veto',     (extrajet2.counts<2))
        cutflow2.addRow( 'minmth>200',   (mth2.min()>200) )

        htag_selection2 = copy.deepcopy(cutflow2.selection)
        
        
        cutflow3 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow3.addRow( 'filters',    filters )
        cutflow3.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow3.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow3.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow3.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline3 = copy.deepcopy(cutflow3.selection)
        
        cutflow3.addRow( 'N_fatjet>1',      (fatjet3.counts>1) )
        cutflow3.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet43>0.5))
        cutflow3.addRow( 'dphiDiFatJet', (dphiDiFatJet3<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD3 = copy.deepcopy(cutflow3.selection)
        
        cutflow3.addRow( 'N_htag>0',     (htag_sel3))
        cutflow3.addRow( 'njet veto',     (extrajet3.counts<2))
        cutflow3.addRow( 'minmth>200',   (mth3.min()>200) )

        htag_selection3 = copy.deepcopy(cutflow3.selection)
        
        
        cutflow4 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow4.addRow( 'filters',    filters )
        cutflow4.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow4.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow4.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow4.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline4 = copy.deepcopy(cutflow4.selection)
        
        cutflow4.addRow( 'N_fatjet>1',      (fatjet4.counts>1) )
        cutflow4.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet44>0.5))
        cutflow4.addRow( 'dphiDiFatJet', (dphiDiFatJet4<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD4 = copy.deepcopy(cutflow4.selection)
        
        cutflow4.addRow( 'N_htag>0',     (htag_sel4))
        cutflow4.addRow( 'njet veto',     (extrajet4.counts<2))
        cutflow4.addRow( 'minmth>200',   (mth4.min()>200) )

        htag_selection4 = copy.deepcopy(cutflow4.selection)
        
        
        cutflow5 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow5.addRow( 'filters',    filters )
        cutflow5.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow5.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow5.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow5.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline5 = copy.deepcopy(cutflow5.selection)
        
        cutflow5.addRow( 'N_fatjet>1',      (fatjet5.counts>1) )
        cutflow5.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet45>0.5))
        cutflow5.addRow( 'dphiDiFatJet', (dphiDiFatJet5<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD5 = copy.deepcopy(cutflow5.selection)
        
        cutflow5.addRow( 'N_htag>0',     (htag_sel5))
        cutflow5.addRow( 'njet veto',     (extrajet5.counts<2))
        cutflow5.addRow( 'minmth>200',   (mth5.min()>200) )

        htag_selection5 = copy.deepcopy(cutflow5.selection)
        
        
        cutflow6 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow6.addRow( 'filters',    filters )
        cutflow6.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow6.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow6.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow6.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline6 = copy.deepcopy(cutflow6.selection)
        
        cutflow6.addRow( 'N_fatjet>1',      (fatjet6.counts>1) )
        cutflow6.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet46>0.5))
        cutflow6.addRow( 'dphiDiFatJet', (dphiDiFatJet6<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets

        vetoQCD6 = copy.deepcopy(cutflow6.selection)
        
        cutflow6.addRow( 'N_htag>0',     (htag_sel6))
        cutflow6.addRow( 'njet veto',     (extrajet6.counts<2))
        cutflow6.addRow( 'minmth>200',   (mth6.min()>200) )

        htag_selection6 = copy.deepcopy(cutflow6.selection)
        
        
        cutflow7 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow7.addRow( 'filters',    filters )
        cutflow7.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow7.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow7.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow7.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline7 = copy.deepcopy(cutflow7.selection)
        
        cutflow7.addRow( 'N_fatjet>1',      (fatjet7.counts>1) )
        cutflow7.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet47>0.5))
        cutflow7.addRow( 'dphiDiFatJet', (dphiDiFatJet7<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD7 = copy.deepcopy(cutflow7.selection)
        
        cutflow7.addRow( 'N_htag>0',     (htag_sel7))
        cutflow7.addRow( 'njet veto',     (extrajet7.counts<2))
        cutflow7.addRow( 'minmth>200',   (mth7.min()>200) )

        htag_selection7 = copy.deepcopy(cutflow7.selection)
        
        
        cutflow8 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow8.addRow( 'filters',    filters )
        cutflow8.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow8.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow8.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow8.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline8 = copy.deepcopy(cutflow8.selection)
        
        cutflow8.addRow( 'N_fatjet>1',      (fatjet8.counts>1) )
        cutflow8.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet48>0.5))
        cutflow8.addRow( 'dphiDiFatJet', (dphiDiFatJet8<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD8 = copy.deepcopy(cutflow8.selection)
        
        cutflow8.addRow( 'N_htag>0',     (htag_sel8))
        cutflow8.addRow( 'njet veto',     (extrajet8.counts<2))
        cutflow8.addRow( 'minmth>200',   (mth8.min()>200) )

        htag_selection8 = copy.deepcopy(cutflow8.selection)
        
        
        cutflow9 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow9.addRow( 'filters',    filters )
        cutflow9.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow9.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow9.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow9.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline9 = copy.deepcopy(cutflow9.selection)
        
        cutflow9.addRow( 'N_fatjet>1',      (fatjet9.counts>1) )
        cutflow9.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet49>0.5))
        cutflow9.addRow( 'dphiDiFatJet', (dphiDiFatJet9<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets

        vetoQCD9 = copy.deepcopy(cutflow9.selection)
        
        cutflow9.addRow( 'N_htag>0',     (htag_sel9))
        cutflow9.addRow( 'njet veto',     (extrajet9.counts<2))
        cutflow9.addRow( 'minmth>200',   (mth9.min()>200) )

        htag_selection9 = copy.deepcopy(cutflow9.selection)
        
        
        cutflow10 = Cutflow(output, df, cfg, processes, weight=weight)
        
        cutflow10.addRow( 'filters',    filters )
        cutflow10.addRow( 'triggers',   triggers )
        
        #cutflow.addRow( 'stitch',   (stitchVar ==1) )
        
        cutflow10.addRow( 'skim',        ((met_pt>200) & (skimjet.counts>1)) )
        #cutflow.addRow( 'Exactly 1 e or mu',   ((electron_med.counts+muon_med.counts)==1) )
        cutflow10.addRow( 'E/mu veto',   ((electron_veto.counts+muon_veto.counts)==1) )
        cutflow10.addRow( 'MET>250',     (met_pt>250) )
        #cutflow.addRow( 'nJet>1',      (jet.counts>1) )
        #cutflow.addRow( 'nBTag>0',     (btag.counts>0) )
        
        baseline10 = copy.deepcopy(cutflow10.selection)
        
        cutflow10.addRow( 'N_fatjet>1',      (fatjet10.counts>1) )
        cutflow10.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet410>0.5))
        cutflow10.addRow( 'dphiDiFatJet', (dphiDiFatJet10<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD10 = copy.deepcopy(cutflow10.selection)
        
        cutflow10.addRow( 'N_htag>0',     (htag_sel10))
        cutflow10.addRow( 'njet veto',     (extrajet10.counts<2))
        cutflow10.addRow( 'minmth>200',   (mth10.min()>200) )

        htag_selection10 = copy.deepcopy(cutflow10.selection)
      
        
        ### And fill the histograms
        output['lead_AK8_pts1'].fill(objecto='pt', pt=leadingFatJet1[htag_selection1].pt.flatten(), weight=fullweight[htag_selection1])
        output['sublead_AK8_pts1'].fill(objecto='pt', pt=subleadingFatJet1[htag_selection1].pt.flatten(), weight=fullweight[htag_selection1])
        
        output['lead_AK8_msoftdrops1'].fill(objecto='sd mass', mass=leadingFatJet1[htag_selection1].msoftdrop.flatten(), weight=fullweight[htag_selection1])
        output['sublead_AK8_msoftdrops1'].fill(objecto='sd mass', mass=subleadingFatJet1[htag_selection1].msoftdrop.flatten(), weight=fullweight[htag_selection1])        
        
        
        output['lead_AK8_pts2'].fill(objecto='pt nom', pt=leadingFatJet2[htag_selection2].pt.flatten(), weight=fullweight[htag_selection2])
        output['sublead_AK8_pts2'].fill(objecto='pt nom', pt=subleadingFatJet2[htag_selection2].pt.flatten(), weight=fullweight[htag_selection2])
        
        output['lead_AK8_msoftdrops2'].fill(objecto='sd mass nom', mass=leadingFatJet2[htag_selection2].msoftdrop.flatten(), weight=fullweight[htag_selection2])
        output['sublead_AK8_msoftdrops2'].fill(objecto='sd mass nom', mass=subleadingFatJet2[htag_selection2].msoftdrop.flatten(), weight=fullweight[htag_selection2])        

      
        output['lead_AK8_msoftdrops3'].fill(objecto='sd mass jms up', mass=leadingFatJet3[htag_selection3].msoftdrop.flatten(), weight=fullweight[htag_selection3])
        output['sublead_AK8_msoftdrops3'].fill(objecto='sd mass jms up', mass=subleadingFatJet3[htag_selection3].msoftdrop.flatten(), weight=fullweight[htag_selection3])        

 
        output['lead_AK8_msoftdrops4'].fill(objecto='sd mass jms down', mass=leadingFatJet4[htag_selection4].msoftdrop.flatten(), weight=fullweight[htag_selection4])
        output['sublead_AK8_msoftdrops4'].fill(objecto='sd mass jms down', mass=subleadingFatJet4[htag_selection4].msoftdrop.flatten(), weight=fullweight[htag_selection4])        

        
        output['lead_AK8_msoftdrops5'].fill(objecto='sd mass jmr up', mass=leadingFatJet5[htag_selection5].msoftdrop.flatten(), weight=fullweight[htag_selection5])
        output['sublead_AK8_msoftdrops5'].fill(objecto='sd mass jmr up', mass=subleadingFatJet5[htag_selection5].msoftdrop.flatten(), weight=fullweight[htag_selection5])        

        
        output['lead_AK8_msoftdrops6'].fill(objecto='sd mass jmr down', mass=leadingFatJet6[htag_selection6].msoftdrop.flatten(), weight=fullweight[htag_selection6])
        output['sublead_AK8_msoftdrops6'].fill(objecto='sd mass jmr down', mass=subleadingFatJet6[htag_selection6].msoftdrop.flatten(), weight=fullweight[htag_selection6])        

        
        output['lead_AK8_pts7'].fill(objecto='pt jer up', pt=leadingFatJet7[htag_selection7].pt.flatten(), weight=fullweight[htag_selection7])
        output['sublead_AK8_pts7'].fill(objecto='pt jer up', pt=subleadingFatJet7[htag_selection7].pt.flatten(), weight=fullweight[htag_selection7])
        
        output['lead_AK8_msoftdrops7'].fill(objecto='sd mass jer up', mass=leadingFatJet7[htag_selection7].msoftdrop.flatten(), weight=fullweight[htag_selection7])
        output['sublead_AK8_msoftdrops7'].fill(objecto='sd mass jer up', mass=subleadingFatJet7[htag_selection7].msoftdrop.flatten(), weight=fullweight[htag_selection7])        

        
        output['lead_AK8_pts8'].fill(objecto='pt jer down', pt=leadingFatJet8[htag_selection8].pt.flatten(), weight=fullweight[htag_selection8])
        output['sublead_AK8_pts8'].fill(objecto='pt jer down', pt=subleadingFatJet8[htag_selection8].pt.flatten(), weight=fullweight[htag_selection8])
        
        output['lead_AK8_msoftdrops8'].fill(objecto='sd mass jer down', mass=leadingFatJet8[htag_selection8].msoftdrop.flatten(), weight=fullweight[htag_selection8])
        output['sublead_AK8_msoftdrops8'].fill(objecto='sd mass jer down', mass=subleadingFatJet8[htag_selection8].msoftdrop.flatten(), weight=fullweight[htag_selection8])        

        
        output['lead_AK8_pts9'].fill(objecto='pt jes total up', pt=leadingFatJet9[htag_selection9].pt.flatten(), weight=fullweight[htag_selection9])
        output['sublead_AK8_pts9'].fill(objecto='pt jes total up', pt=subleadingFatJet9[htag_selection9].pt.flatten(), weight=fullweight[htag_selection9])
        
        output['lead_AK8_msoftdrops9'].fill(objecto='sd mass jes total up', mass=leadingFatJet9[htag_selection9].msoftdrop.flatten(), weight=fullweight[htag_selection9])
        output['sublead_AK8_msoftdrops9'].fill(objecto='sd mass jes total up', mass=subleadingFatJet9[htag_selection9].msoftdrop.flatten(), weight=fullweight[htag_selection9])        

        
        output['lead_AK8_pts10'].fill(objecto='pt jes total down', pt=leadingFatJet10[htag_selection10].pt.flatten(), weight=fullweight[htag_selection10])
        output['sublead_AK8_pts10'].fill(objecto='pt jes total down', pt=subleadingFatJet10[htag_selection10].pt.flatten(), weight=fullweight[htag_selection10])
        
        output['lead_AK8_msoftdrops10'].fill(objecto='sd mass jes total down', mass=leadingFatJet10[htag_selection10].msoftdrop.flatten(), weight=fullweight[htag_selection10])
        output['sublead_AK8_msoftdrops10'].fill(objecto='sd mass jes total down', mass=subleadingFatJet10[htag_selection10].msoftdrop.flatten(), weight=fullweight[htag_selection10])        

        return output

    def postprocess(self, accumulator):
        return accumulator

In [6]:
runLocal = True

#scheduler_address = get_scheduler_address()

if not runLocal:
    # Get the scheduler from the dask_cluster notebook
    from dask.distributed import Client, progress

    c = Client(scheduler_address)

    ## for dask
    exe_args = {
        'client': c,
        #'savemetrics': True,
    }
    exe = processor.dask_executor
    
else:
    ## for local
    exe_args = {
        'workers': 4,
        'function_args': {'flatten': False}
    }
    exe = processor.futures_executor

if not runLocal:
    print(c)

In [8]:
overwrite = True
small = False

from Tools.WH_samples import *

if year == 2016:
    fileset_year = fileset_2016
    TChiWH_1000_1 = glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/v0.2.4/Summer16_TChiWH_1000_1.root')
elif year == 2017:
    fileset_year = fileset_2017
    TChiWH_1000_1 = glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/v0.2.4/Fall17_TChiWH_1000_1.root')
elif year == 2018:
    fileset_year = fileset_2018
    TChiWH_1000_1 = glob.glob('/hadoop/cms/store/user/dspitzba/WH_hadronic/v0.2.4/Autumn18_TChiWH_1000_1.root')
    full_sim = glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/v0.2.4/ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8_RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/*.root')


fileset   = {#'WJets': fileset_year['WJets'],
                #'TTJets': fileset_year['TTJets'],
                #'ST': fileset_year['ST'],
                #'WW': fileset_year['WW'],
                #'ttW': fileset_year['TTW'],
                #'QCD': fileset_year['QCD'],
                #'ZNuNu': fileset_year['ZNuNu'] + fileset_year['TTZ'] + fileset_year['VV'],
                #'Data': fileset_year['MET'],
                'full_sim': full_sim,
                }




# load the config and the cache
cfg = loadConfig()

cacheName = 'WH_LL_%s_small'%year if small else 'WH_LL_%s'%year

## WH_LL cache contains the plots for "resolved" like CRs

# histograms
#histograms = []
#histograms += ['N_AK4']

# initialize cache
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)
if not overwrite:
    cache.load()

'''and histograms == cache.get('histograms')'''
if cfg == cache.get('cfg') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    # Run the processor
    if small:
        fileset = fileset_small
        exe_args['workers'] = 4
    else:
        fileset = fileset
        exe_args['workers'] = 16
    
        
    output = processor.run_uproot_job(fileset,
                                      treename='Events',
                                      processor_instance=analysisProcessor(),
                                      executor=exe,
                                      executor_args=exe_args,
                                      #chunksize=250000,
                                      chunksize=100000,
                                     )
    cache['fileset']        = fileset
    cache['cfg']            = cfg
    #cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

HBox(children=(HTML(value='Preprocessing'), FloatProgress(value=0.0, max=12.0), HTML(value='')))

HBox(children=(HTML(value='Processing'), FloatProgress(value=0.0, max=15.0), HTML(value='')))

/home/users/ewallace/CMSSW_10_2_9/src/tW_scattering2/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/ewallace/CMSSW_10_2_9/src/tW_scattering2/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/ewallace/CMSSW_10_2_9/src/tW_scattering2/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/ewallace/CMSSW_10_2_9/src/tW_scattering2/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/ewallace/CMSSW_10_2_9/src/tW_scattering2/coffeaEnv/lib/python3.6/site-packages/awkwa

In [9]:
# Cutflow
from Tools.helpers import getCutFlowTable

linesList= ['filters', 'triggers', 'skim',  'E/mu veto', 'MET>250', 'N_fatjet>1', 'min_dphiFatJetMet4', 'dphiDiFatJet', 'N_htag>0', 'njet veto', 'minmth>200', 'N_wtag>0', 'N_htag>0, N_wtag>0']
#linesList= ['filters_VR', 'triggers_VR', 'E/mu veto_VR', 'MET>250_VR', 'N_fatjet>1_VR', 'min_dphiFatJetMet4_VR', 'dphiDiFatJet_VR', 'N_htag>0_VR', 'N_wtag>0_VR']


processes = processesList
lines     = ['entry']
lines    += linesList
df        = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=4, signal='full_sim')
df

,full_sim,WJets,TTJets,ST,ttW,WW,ZNuNu,QCD,Data,S/B
entry,17280.0 +/- 10.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
filters,1737.0 +/- 2.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
triggers,1737.0 +/- 2.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
skim,504.3 +/- 1.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
E/mu veto,248.2 +/- 0.7,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
MET>250,112.4 +/- 0.5,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
N_fatjet>1,55.51 +/- 0.33,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
min_dphiFatJetMet4,28.3 +/- 0.23,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
dphiDiFatJet,22.69 +/- 0.2,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1
N_htag>0,15.95 +/- 0.17,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,0.0 +/- 0.0,1


In [10]:
from plots.helpers import *

useData         = False
normalize       = False

In [11]:
bins1 = {\
    'lead_AK8_pts1': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('pt', r'$pt$ $(lead. Htag) \ (GeV)$', 32, 200, 1000)},
    'lead_AK8_pts2': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(lead. Htag) \ (GeV)$', 32, 200, 1000)},
    'lead_AK8_pts7': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(lead. Htag) \ (GeV)$', 32, 200, 1000)},
    'lead_AK8_pts8': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(lead. Htag) \ (GeV)$', 32, 200, 1000)},
    'lead_AK8_pts9': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(lead. Htag) \ (GeV)$', 32, 200, 1000)},
    'lead_AK8_pts10': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(lead. Htag) \ (GeV)$', 32, 200, 1000)},
    }

In [15]:
# Make some of the plots

plotDir = '/home/users/ewallace/public_html/sdplots/'
finalizePlotDir(plotDir)

#if not histograms:
#    print ("Couldn't find histograms in archive. Quitting.")
#    exit()

print ("Plots will appear here:", plotDir )

histogram1 = output['lead_AK8_pts1']
histogram2 = output['lead_AK8_pts2']
histogram3 = output['lead_AK8_pts7']
histogram4 = output['lead_AK8_pts8']
histogram5 = output['lead_AK8_pts9']
histogram6 = output['lead_AK8_pts10']
    

axis1 = bins1['lead_AK8_pts1']['axis']
print ('lead_AK8_pts1', axis1)
histogram1 = histogram1.rebin(axis1, bins1['lead_AK8_pts1']['bins'])

axis2 = bins1['lead_AK8_pts2']['axis']
print ('lead_AK8_pts2', axis2)
histogram2 = histogram2.rebin(axis2, bins1['lead_AK8_pts2']['bins'])

axis3 = bins1['lead_AK8_pts7']['axis']
print ('lead_AK8_pts7', axis3)
histogram3 = histogram3.rebin(axis3, bins1['lead_AK8_pts7']['bins'])

axis4 = bins1['lead_AK8_pts8']['axis']
print ('lead_AK8_pts8', axis4)
histogram4 = histogram4.rebin(axis4, bins1['lead_AK8_pts8']['bins'])

axis5 = bins1['lead_AK8_pts9']['axis']
print ('lead_AK8_pts9', axis5)
histogram5 = histogram5.rebin(axis5, bins1['lead_AK8_pts9']['bins'])

axis6 = bins1['lead_AK8_pts10']['axis']
print ('lead_AK8_pts10', axis6)
histogram6 = histogram6.rebin(axis6, bins1['lead_AK8_pts10']['bins'])

y_max = histogram1.sum("objecto").values(overflow='over')[()].max()
y_over = histogram1.sum("objecto").values(overflow='over')[()][-1]

if useData:
        fig, (ax, rax) = plt.subplots(2, 1, figsize=(7,7), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
else:
        fig, ax = plt.subplots(1,1,figsize=(7,7))

    # get axes
    
hist.plot1d(histogram1, overlay="objecto", ax=ax, overflow=bins1['lead_AK8_pts1']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram2, overlay="objecto", ax=ax, overflow=bins1['lead_AK8_pts2']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram3, overlay="objecto", ax=ax, overflow=bins1['lead_AK8_pts7']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram4, overlay="objecto", ax=ax, overflow=bins1['lead_AK8_pts8']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram5, overlay="objecto", ax=ax, overflow=bins1['lead_AK8_pts9']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram6, overlay="objecto", ax=ax, overflow=bins1['lead_AK8_pts10']['overflow'], line_opts={'linewidth': 3}, clear = False)

    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, order=processes)
    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, error_opts=error_opts, order=processes)
    #hist.plot1d(histogram['QCD'], overlay="dataset", ax=ax, stack=False, overflow=bins[name]['overflow'], clear=False, line_opts=None, fill_opts=fill_opts, error_opts=error_opts, order=processes)
if useData:
    hist.plot1d(histogram['Data'], overlay="objecto", ax=ax, overflow=bins1['lead_AK8_pts1']['overflow'], error_opts=data_err_opts, clear=False)
    #hist.plot1d(histogram[signal], overlay="dataset", ax=ax, overflow=bins[name]['overflow'], line_opts={'linewidth':3}, clear=False)

if useData:
    # build ratio
    hist.plotratio(
            num=histogram['Data'].sum("dataset"),
            denom=histogram[notdata].sum("dataset"),
            ax=rax,
            error_opts=data_err_opts,
            denom_fill_opts=None, # triggers this: https://github.com/CoffeaTeam/coffea/blob/master/coffea/hist/plot.py#L376
            guide_opts={},
            unc='num',
            #unc=None,
            overflow=bins[name]['overflow']
        )


for l in ['linear', 'log']:
    if useData:
        saveFig(fig, ax, rax, plotDir, name, scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
    else:
        saveFig(fig, ax, None, plotDir, name='lead_htag_pt_fullsim_2018', scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
fig.clear()
if useData:
    rax.clear()
    ax.clear()


Plots will appear here: /home/users/ewallace/public_html/sdplots/
lead_AK8_pts1 pt
lead_AK8_pts2 pt
lead_AK8_pts7 pt
lead_AK8_pts8 pt
lead_AK8_pts9 pt
lead_AK8_pts10 pt


<Figure size 504x504 with 0 Axes>

In [13]:
bins2 = {\
    'sublead_AK8_pts1': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('pt', r'$pt$ $(sublead. Htag) \ (GeV)$', 32, 200, 1000)},
    'sublead_AK8_pts2': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(sublead. Htag) \ (GeV)$', 32, 200, 1000)},
    'sublead_AK8_pts7': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(sublead. Htag) \ (GeV)$', 32, 200, 1000)},
    'sublead_AK8_pts8': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(sublead. Htag) \ (GeV)$', 32, 200, 1000)},
    'sublead_AK8_pts9': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(sublead. Htag) \ (GeV)$', 32, 200, 1000)},
    'sublead_AK8_pts10': {'axis': 'pt', 'overflow':'over', 'bins': hist.Bin('mass', r'$pt$ $(sublead. Htag) \ (GeV)$', 32, 200, 1000)},
    }

In [16]:
# Make some of the plots

plotDir = '/home/users/ewallace/public_html/sdplots/'
finalizePlotDir(plotDir)

#if not histograms:
#    print ("Couldn't find histograms in archive. Quitting.")
#    exit()

print ("Plots will appear here:", plotDir )

histogram1 = output['sublead_AK8_pts1']
histogram2 = output['sublead_AK8_pts2']
histogram3 = output['sublead_AK8_pts7']
histogram4 = output['sublead_AK8_pts8']
histogram5 = output['sublead_AK8_pts9']
histogram6 = output['sublead_AK8_pts10']
    

axis1 = bins2['sublead_AK8_pts1']['axis']
print ('sublead_AK8_pts1', axis1)
histogram1 = histogram1.rebin(axis1, bins2['sublead_AK8_pts1']['bins'])

axis2 = bins2['sublead_AK8_pts2']['axis']
print ('sublead_AK8_pts2', axis2)
histogram2 = histogram2.rebin(axis2, bins2['sublead_AK8_pts2']['bins'])

axis3 = bins2['sublead_AK8_pts7']['axis']
print ('sublead_AK8_pts7', axis3)
histogram3 = histogram3.rebin(axis3, bins2['sublead_AK8_pts7']['bins'])

axis4 = bins2['sublead_AK8_pts8']['axis']
print ('sublead_AK8_pts8', axis4)
histogram4 = histogram4.rebin(axis4, bins2['sublead_AK8_pts8']['bins'])

axis5 = bins2['sublead_AK8_pts9']['axis']
print ('sublead_AK8_pts9', axis5)
histogram5 = histogram5.rebin(axis5, bins2['sublead_AK8_pts9']['bins'])

axis6 = bins2['sublead_AK8_pts10']['axis']
print ('sublead_AK8_pts10', axis6)
histogram6 = histogram6.rebin(axis6, bins2['sublead_AK8_pts10']['bins'])
    
y_max = histogram2.sum("objecto").values(overflow='over')[()].max()
y_over = histogram2.sum("objecto").values(overflow='over')[()][-1]    
       
if useData:
    fig, (ax, rax) = plt.subplots(2, 1, figsize=(7,7), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
else:
    fig, ax = plt.subplots(1,1,figsize=(7,7))

# get axes
    
hist.plot1d(histogram1, overlay="objecto", ax=ax, overflow=bins2['sublead_AK8_pts1']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram2, overlay="objecto", ax=ax, overflow=bins2['sublead_AK8_pts2']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram3, overlay="objecto", ax=ax, overflow=bins2['sublead_AK8_pts7']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram4, overlay="objecto", ax=ax, overflow=bins2['sublead_AK8_pts8']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram5, overlay="objecto", ax=ax, overflow=bins2['sublead_AK8_pts9']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram6, overlay="objecto", ax=ax, overflow=bins2['sublead_AK8_pts10']['overflow'], line_opts={'linewidth': 3}, clear = False)
    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, order=processes)
    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, error_opts=error_opts, order=processes)
    #hist.plot1d(histogram['QCD'], overlay="dataset", ax=ax, stack=False, overflow=bins[name]['overflow'], clear=False, line_opts=None, fill_opts=fill_opts, error_opts=error_opts, order=processes)
if useData:
    hist.plot1d(histogram['Data'], overlay="objecto", ax=ax, overflow=bins[name]['overflow'], error_opts=data_err_opts, clear=False)
    #hist.plot1d(histogram[signal], overlay="dataset", ax=ax, overflow=bins[name]['overflow'], line_opts={'linewidth':3}, clear=False)

if useData:
    # build ratio
    hist.plotratio(
            num=histogram['Data'].sum("dataset"),
            denom=histogram[notdata].sum("dataset"),
            ax=rax,
            error_opts=data_err_opts,
            denom_fill_opts=None, # triggers this: https://github.com/CoffeaTeam/coffea/blob/master/coffea/hist/plot.py#L376
            guide_opts={},
            unc='num',
            #unc=None,
            overflow=bins[name]['overflow']
        )


for l in ['linear', 'log']:
    if useData:
        saveFig(fig, ax, rax, plotDir, name, scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
    else:
        saveFig(fig, ax, None, plotDir, name='sublead_htag_pt_fullsim_2018', scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
fig.clear()
if useData:
    rax.clear()
    ax.clear()


Plots will appear here: /home/users/ewallace/public_html/sdplots/
sublead_AK8_pts1 pt
sublead_AK8_pts2 pt
sublead_AK8_pts7 pt
sublead_AK8_pts8 pt
sublead_AK8_pts9 pt
sublead_AK8_pts10 pt


<Figure size 504x504 with 0 Axes>

In [17]:
bins3 = {\
    'lead_AK8_msoftdrops1': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops2': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops3': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops4': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops5': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops6': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops7': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops8': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops9': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
    'lead_AK8_msoftdrops10': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(lead. Htag) \ (GeV)$', 25, 0, 250)},
        }

In [18]:
# Make some of the plots

plotDir = '/home/users/ewallace/public_html/sdplots/'
finalizePlotDir(plotDir)

#if not histograms:
#    print ("Couldn't find histograms in archive. Quitting.")
#    exit()

print ("Plots will appear here:", plotDir )

histogram1 = output['lead_AK8_msoftdrops1']
histogram2 = output['lead_AK8_msoftdrops2']
histogram3 = output['lead_AK8_msoftdrops3']
histogram4 = output['lead_AK8_msoftdrops4']
histogram5 = output['lead_AK8_msoftdrops5']
histogram6 = output['lead_AK8_msoftdrops6']
histogram7 = output['lead_AK8_msoftdrops7']
histogram8 = output['lead_AK8_msoftdrops8']
histogram9 = output['lead_AK8_msoftdrops9']
histogram10 = output['lead_AK8_msoftdrops10']

    

axis1 = bins3['lead_AK8_msoftdrops1']['axis']
print ('lead_AK8_msoftdrops1', axis1)
histogram1 = histogram1.rebin(axis1, bins3['lead_AK8_msoftdrops1']['bins'])

axis2 = bins3['lead_AK8_msoftdrops2']['axis']
print ('lead_AK8_msoftdrops2', axis2)
histogram2 = histogram2.rebin(axis2, bins3['lead_AK8_msoftdrops2']['bins'])

axis3 = bins3['lead_AK8_msoftdrops3']['axis']
print ('lead_AK8_msoftdrops3', axis3)
histogram3 = histogram3.rebin(axis3, bins3['lead_AK8_msoftdrops3']['bins'])

axis4 = bins3['lead_AK8_msoftdrops4']['axis']
print ('lead_AK8_msoftdrops1', axis4)
histogram4 = histogram4.rebin(axis4, bins3['lead_AK8_msoftdrops4']['bins'])

axis5 = bins3['lead_AK8_msoftdrops5']['axis']
print ('lead_AK8_msoftdrops5', axis5)
histogram5 = histogram5.rebin(axis5, bins3['lead_AK8_msoftdrops5']['bins'])

axis6 = bins3['lead_AK8_msoftdrops6']['axis']
print ('lead_AK8_msoftdrops1', axis6)
histogram6 = histogram6.rebin(axis6, bins3['lead_AK8_msoftdrops6']['bins'])

axis7 = bins3['lead_AK8_msoftdrops7']['axis']
print ('lead_AK8_msoftdrops7', axis1)
histogram7 = histogram7.rebin(axis7, bins3['lead_AK8_msoftdrops7']['bins'])

axis8 = bins3['lead_AK8_msoftdrops8']['axis']
print ('lead_AK8_msoftdrops8', axis8)
histogram8 = histogram8.rebin(axis8, bins3['lead_AK8_msoftdrops8']['bins'])

axis9 = bins3['lead_AK8_msoftdrops9']['axis']
print ('lead_AK8_msoftdrops9', axis9)
histogram9 = histogram9.rebin(axis1, bins3['lead_AK8_msoftdrops9']['bins'])

axis10 = bins3['lead_AK8_msoftdrops10']['axis']
print ('lead_AK8_msoftdrops10', axis10)
histogram10 = histogram10.rebin(axis10, bins3['lead_AK8_msoftdrops10']['bins'])


y_max = histogram2.sum("objecto").values(overflow='over')[()].max()
y_over = histogram2.sum("objecto").values(overflow='over')[()][-1]


if useData:
        fig, (ax, rax) = plt.subplots(2, 1, figsize=(7,7), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
else:
        fig, ax = plt.subplots(1,1,figsize=(7,7))

    # get axes
    
hist.plot1d(histogram1, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops1']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram2, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops2']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram3, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops3']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram4, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops4']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram5, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops5']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram6, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops6']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram7, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops7']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram8, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops8']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram9, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops9']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram10, overlay="objecto", ax=ax, overflow=bins3['lead_AK8_msoftdrops10']['overflow'], line_opts={'linewidth': 3}, clear = False)

    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, order=processes)
    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, error_opts=error_opts, order=processes)
    #hist.plot1d(histogram['QCD'], overlay="dataset", ax=ax, stack=False, overflow=bins[name]['overflow'], clear=False, line_opts=None, fill_opts=fill_opts, error_opts=error_opts, order=processes)
if useData:
    hist.plot1d(histogram['Data'], overlay="objecto", ax=ax, overflow=bins[name]['overflow'], error_opts=data_err_opts, clear=False)
    #hist.plot1d(histogram[signal], overlay="dataset", ax=ax, overflow=bins[name]['overflow'], line_opts={'linewidth':3}, clear=False)

if useData:
    # build ratio
    hist.plotratio(
            num=histogram['Data'].sum("dataset"),
            denom=histogram[notdata].sum("dataset"),
            ax=rax,
            error_opts=data_err_opts,
            denom_fill_opts=None, # triggers this: https://github.com/CoffeaTeam/coffea/blob/master/coffea/hist/plot.py#L376
            guide_opts={},
            unc='num',
            #unc=None,
            overflow=bins[name]['overflow']
        )


for l in ['linear', 'log']:
    if useData:
        saveFig(fig, ax, rax, plotDir, name, scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
    else:
        saveFig(fig, ax, None, plotDir, name='lead_htag_msoftdrop_fullsim_2018', scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
fig.clear()
if useData:
    rax.clear()
    ax.clear()


Plots will appear here: /home/users/ewallace/public_html/sdplots/
lead_AK8_msoftdrops1 mass
lead_AK8_msoftdrops2 mass
lead_AK8_msoftdrops3 mass
lead_AK8_msoftdrops1 mass
lead_AK8_msoftdrops5 mass
lead_AK8_msoftdrops1 mass
lead_AK8_msoftdrops7 mass
lead_AK8_msoftdrops8 mass
lead_AK8_msoftdrops9 mass
lead_AK8_msoftdrops10 mass


<Figure size 504x504 with 0 Axes>

In [20]:
bins4 = {\
    'sublead_AK8_msoftdrops1': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops2': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops3': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops4': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops5': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops6': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops7': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops8': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops9': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
    'sublead_AK8_msoftdrops10': {'axis': 'mass', 'overflow':'over', 'bins': hist.Bin('mass', r'$sd mass$ $(sublead. Htag) \ (GeV)$', 25, 0, 250)},
        }

In [21]:
# Make some of the plots

plotDir = '/home/users/ewallace/public_html/sdplots/'
finalizePlotDir(plotDir)

#if not histograms:
#    print ("Couldn't find histograms in archive. Quitting.")
#    exit()

print ("Plots will appear here:", plotDir )

histogram1 = output['sublead_AK8_msoftdrops1']
histogram2 = output['sublead_AK8_msoftdrops2']
histogram3 = output['sublead_AK8_msoftdrops3']
histogram4 = output['sublead_AK8_msoftdrops4']
histogram5 = output['sublead_AK8_msoftdrops5']
histogram6 = output['sublead_AK8_msoftdrops6']
histogram7 = output['sublead_AK8_msoftdrops7']
histogram8 = output['sublead_AK8_msoftdrops8']
histogram9 = output['sublead_AK8_msoftdrops9']
histogram10 = output['sublead_AK8_msoftdrops10']

    

axis1 = bins4['sublead_AK8_msoftdrops1']['axis']
print ('sublead_AK8_msoftdrops1', axis1)
histogram1 = histogram1.rebin(axis1, bins4['sublead_AK8_msoftdrops1']['bins'])

axis2 = bins4['sublead_AK8_msoftdrops2']['axis']
print ('sublead_AK8_msoftdrops2', axis1)
histogram2 = histogram2.rebin(axis2, bins4['sublead_AK8_msoftdrops2']['bins'])

axis3 = bins4['sublead_AK8_msoftdrops3']['axis']
print ('sublead_AK8_msoftdrops3', axis1)
histogram3 = histogram3.rebin(axis3, bins4['sublead_AK8_msoftdrops3']['bins'])

axis4 = bins4['sublead_AK8_msoftdrops4']['axis']
print ('sublead_AK8_msoftdrops1', axis4)
histogram4 = histogram4.rebin(axis4, bins4['sublead_AK8_msoftdrops4']['bins'])

axis5 = bins4['sublead_AK8_msoftdrops5']['axis']
print ('sublead_AK8_msoftdrops5', axis5)
histogram5 = histogram5.rebin(axis5, bins4['sublead_AK8_msoftdrops5']['bins'])

axis6 = bins4['sublead_AK8_msoftdrops6']['axis']
print ('sublead_AK8_msoftdrops1', axis6)
histogram6 = histogram6.rebin(axis6, bins4['sublead_AK8_msoftdrops6']['bins'])

axis7 = bins4['sublead_AK8_msoftdrops7']['axis']
print ('sublead_AK8_msoftdrops7', axis1)
histogram7 = histogram7.rebin(axis7, bins4['sublead_AK8_msoftdrops7']['bins'])

axis8 = bins4['sublead_AK8_msoftdrops8']['axis']
print ('sublead_AK8_msoftdrops8', axis8)
histogram8 = histogram8.rebin(axis8, bins4['sublead_AK8_msoftdrops8']['bins'])

axis9 = bins4['sublead_AK8_msoftdrops9']['axis']
print ('sublead_AK8_msoftdrops9', axis9)
histogram9 = histogram9.rebin(axis1, bins4['sublead_AK8_msoftdrops9']['bins'])

axis10 = bins4['sublead_AK8_msoftdrops10']['axis']
print ('sublead_AK8_msoftdrops10', axis10)
histogram10 = histogram10.rebin(axis10, bins4['sublead_AK8_msoftdrops10']['bins'])

    

if useData:
        fig, (ax, rax) = plt.subplots(2, 1, figsize=(7,7), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
else:
        fig, ax = plt.subplots(1,1,figsize=(7,7))

    # get axes
    
hist.plot1d(histogram1, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops1']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram2, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops2']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram3, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops3']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram4, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops4']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram5, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops5']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram6, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops6']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram7, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops7']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram8, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops8']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram9, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops9']['overflow'], line_opts={'linewidth': 3}, clear = False)
hist.plot1d(histogram10, overlay="objecto", ax=ax, overflow=bins4['sublead_AK8_msoftdrops10']['overflow'], line_opts={'linewidth': 3}, clear = False)

    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, order=processes)
    #hist.plot1d(histogram[notdata], overlay="dataset", ax=ax, stack=True, overflow=bins[name]['overflow'], fill_opts=fill_opts, error_opts=error_opts, order=processes)
    #hist.plot1d(histogram['QCD'], overlay="dataset", ax=ax, stack=False, overflow=bins[name]['overflow'], clear=False, line_opts=None, fill_opts=fill_opts, error_opts=error_opts, order=processes)
if useData:
    hist.plot1d(histogram['Data'], overlay="objecto", ax=ax, overflow=bins[name]['overflow'], error_opts=data_err_opts, clear=False)
    #hist.plot1d(histogram[signal], overlay="dataset", ax=ax, overflow=bins[name]['overflow'], line_opts={'linewidth':3}, clear=False)

if useData:
    # build ratio
    hist.plotratio(
            num=histogram['Data'].sum("dataset"),
            denom=histogram[notdata].sum("dataset"),
            ax=rax,
            error_opts=data_err_opts,
            denom_fill_opts=None, # triggers this: https://github.com/CoffeaTeam/coffea/blob/master/coffea/hist/plot.py#L376
            guide_opts={},
            unc='num',
            #unc=None,
            overflow=bins[name]['overflow']
        )


for l in ['linear', 'log']:
    if useData:
        saveFig(fig, ax, rax, plotDir, name, scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
    else:
        saveFig(fig, ax, None, plotDir, name='sublead_htag_msoftdrop_fullsim_2018', scale=l, shape=False, y_max=y_max, preliminary='Simulation', lumi=lumis[year])
fig.clear()
if useData:
    rax.clear()
    ax.clear()


Plots will appear here: /home/users/ewallace/public_html/sdplots/
sublead_AK8_msoftdrops1 mass
sublead_AK8_msoftdrops2 mass
sublead_AK8_msoftdrops3 mass
sublead_AK8_msoftdrops1 mass
sublead_AK8_msoftdrops5 mass
sublead_AK8_msoftdrops1 mass
sublead_AK8_msoftdrops7 mass
sublead_AK8_msoftdrops8 mass
sublead_AK8_msoftdrops9 mass
sublead_AK8_msoftdrops10 mass


<Figure size 504x504 with 0 Axes>